In [1]:
!nvidia-smi

Sat Apr 15 19:33:59 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.84       Driver Version: 460.84       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:04:00.0 Off |                  N/A |
| 34%   67C    P2    91W / 250W |   8657MiB / 11178MiB |     93%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:06:00.0 Off |                  N/A |
| 38%   

In [2]:
import os
import torch
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, "0" to  "7" 
os.environ["CUDA_VISIBLE_DEVICES"]="5,6,7"
#torch.cuda.set_device(0)
n_gpu = torch.cuda.device_count()
print(n_gpu)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.get_device_name(0)


3


'GeForce GTX 1080 Ti'

In [3]:
import warnings 
warnings.filterwarnings('ignore')
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm


In [4]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation
#Set a seed
import random
import numpy as np
import torch
import datasets
import pandas as pd

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

from torch.utils.data import Dataset, DataLoader

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

[nltk_data] Downloading package punkt to /DATA/gupta92/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

set_seed(42)

In [6]:
#Loading dataset

In [7]:
pd.set_option('display.max_colwidth', None)

In [8]:
df = pd.read_csv('DATA/etoori_train.csv')
df.shape


(140000, 2)

# Training on Hindi_Artificial_train dataset

In [9]:
df = pd.read_csv('DATA/Hindi_Artificial_train.csv')
df.shape

(2592885, 2)

In [10]:
#df = df.head(500000)

In [8]:
import pandas as pd


df_adp_infl = pd.read_csv('DATA/Wikiedits_ADP_INFL.csv')
print("ADP_INFL:", df_adp_infl.shape)
df1 = pd.read_csv('DATA/HiWikEd.csv')
df1 = df1.drop(['Unnamed: 0'], axis=1)
df1.shape

ADP_INFL: (1101, 2)


(13187, 2)

In [9]:
df_adp_infl.head(2)

,enc_input,dec_input
0,जल प्रदूषण पर नियंत्रण हेतु नालों का नियमित रूप से साफ सफाई करना चाहिए.,जल प्रदूषण पर नियंत्रण हेतु नालों की नियमित रूप से साफ सफाई करना चाहिए.
1,ये यूनिसन ग्रुप का सबसे बड़ी योजना थी.,ये यूनिसन ग्रुप की सबसे बड़ी योजना थी.


In [10]:
df1.head(2)

,enc_input,dec_input
0,चाय की दुकान से लेकर वाहनों और दिवारों तक हर जगह विज्ञापन ही विज्ञापन दिखाई देते है.,चाय की दुकान से लेकर वाहनों और दिवारों तक हर जगह विज्ञापन ही विज्ञापन दिखाई देते हैं.
1,"ये कहीं पे निगाहें, कही पे निशाना का सा अन्दाज है.","यह कहीं पे निगाहें, कही पे निशाना का सा अन्दाज है."


In [11]:
frames = [df1, df_adp_infl]
df = pd.concat(frames)
df.shape

(14288, 2)

In [12]:
df.head(2)

,enc_input,dec_input
0,चाय की दुकान से लेकर वाहनों और दिवारों तक हर जगह विज्ञापन ही विज्ञापन दिखाई देते है.,चाय की दुकान से लेकर वाहनों और दिवारों तक हर जगह विज्ञापन ही विज्ञापन दिखाई देते हैं.
1,"ये कहीं पे निगाहें, कही पे निशाना का सा अन्दाज है.","यह कहीं पे निगाहें, कही पे निशाना का सा अन्दाज है."


In [13]:
df.rename(columns = {'enc_input':'input'}, inplace = True)
df.rename(columns = {'dec_input':'output'}, inplace = True)

In [14]:
#df = df.drop(['Unnamed: 0'], axis=1)

In [15]:
df.head(2)

,input,output
0,चाय की दुकान से लेकर वाहनों और दिवारों तक हर जगह विज्ञापन ही विज्ञापन दिखाई देते है.,चाय की दुकान से लेकर वाहनों और दिवारों तक हर जगह विज्ञापन ही विज्ञापन दिखाई देते हैं.
1,"ये कहीं पे निगाहें, कही पे निशाना का सा अन्दाज है.","यह कहीं पे निगाहें, कही पे निशाना का सा अन्दाज है."


In [16]:
from transformers import (
    T5ForConditionalGeneration, T5Tokenizer, 
    Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
  )

from torch.utils.data import Dataset, DataLoader

In [17]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")

In [18]:
tokenizer

PreTrainedTokenizerFast(name_or_path='google/muril-base-cased', vocab_size=197285, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [19]:
tokenizer.model_max_length

512

In [20]:
#model = AutoModelForMaskedLM.from_pretrained("google/muril-base-cased")

In [21]:

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

#model_name = 't5-base'
#tokenizer = T5Tokenizer.from_pretrained(model_name)
#model = T5ForConditionalGeneration.from_pretrained(model_name)

#model_name = 't5_gec_hindi_muRIL'
#model_name = 't5_gec_hindi_muRIL_best'
model_name ='t5_gec_hindi_muRIL_best_Full'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu' 
model = T5ForConditionalGeneration.from_pretrained(model_name).to(torch_device)



In [22]:
def calc_token_len(example):
    return len(tokenizer(example).input_ids)

In [23]:
#Train - Test Split
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.10, shuffle=True)
train_df.shape, test_df.shape

((12859, 2), (1429, 2))

In [26]:
test_df['input_token_len'] = test_df['input'].apply(calc_token_len)

In [27]:
test_df.head()

,input,output,input_token_len
4450,यह वेल्स राष्ट्रीय रग्बी यूनियन टीम का घर है और वेल्स की राष्ट्रीय फ़ुटबॉल टीम के खेल की मेजबानी भी करता है.,यह वेल्स राष्ट्रीय रग्बी यूनियन टीम का घर है और वेल्स की राष्ट्रीय फ़ुटबॉल टीम के मैच की मेजबानी भी करता है.,26
7854,"चक्षुषा हिंदू पौराणिक कथाओं के एक पात्र हैं, जो छठे मन्वंतर के मनु हैं, जो ऋषि वृहती के पुत्र हैं.","चक्षुषा हिंदू पौराणिक कथाओं के एक पात्र हैं, जो छठे मन्वंतर के मनु हैं, तथा ऋषि वृहती के पुत्र हैं.",30
7050,इस गांव में छत्रिय ब्राह्मण दलित बघेल आज जातियां मुख्य रूप से पाई जाती हैं.,इस गांव में छत्रिय ब्राह्मण दलित बघेल आदि जातियां मुख्य रूप से पाई जाती हैं.,21
12868,"यदि इस गांव से ऊपर की ओर जाया जाए तो घोरल, एंटीलोप्स, काले और लाल भालुओं को देखा जा सकता है.","यदि इस ग्राम से ऊपर की ओर जाया जाए तो घोरल, एंटीलोप्स, काले और लाल भालुओं को देखा जा सकता है.",30
7242,वह एक दशक से अधिक समय के लिए एक प्रमुख शौकिया चैंपियन था.,वह एक दशक से अधिक समय के लिए एक प्रधान शौकिया चैंपियन था.,16


In [29]:
test_df['input_token_len'].describe()

train_df = df

In [30]:
#We will use a token length of 64 since it will cover the vast majority of examples
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [31]:
train_df.input.str.len().max()

183

In [32]:
#Load the Dataset
from torch.utils.data import Dataset, DataLoader
class GrammarDataset(Dataset):
    def __init__(self, dataset, tokenizer,print_text=False):         
        self.dataset = dataset
        self.pad_to_max_length = False
        self.tokenizer = tokenizer
        self.print_text = print_text
        #self.max_len = 128
        self.max_len = 64
  
    def __len__(self):
        return len(self.dataset)


    def tokenize_data(self, example):
        input_, target_ = example['input'], example['output']

        # tokenize inputs
        tokenized_inputs = tokenizer(input_, pad_to_max_length=self.pad_to_max_length, 
                                            max_length=self.max_len,
                                            return_attention_mask=True)
    
        tokenized_targets = tokenizer(target_, pad_to_max_length=self.pad_to_max_length, 
                                            max_length=self.max_len,
                                            return_attention_mask=True)

        inputs={"input_ids": tokenized_inputs['input_ids'],
            "attention_mask": tokenized_inputs['attention_mask'],
            "labels": tokenized_targets['input_ids']
        }
        
        return inputs

  
    def __getitem__(self, index):
        inputs = self.tokenize_data(self.dataset[index])
        
        if self.print_text:
            for k in inputs.keys():
                print(k, len(inputs[k]))

        return inputs

In [33]:
dataset = GrammarDataset(test_dataset, tokenizer, True)
print(dataset[10])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


input_ids 13
attention_mask 13
labels 13
{'input_ids': [104, 1503, 23421, 6407, 2768, 2254, 7593, 1124, 2768, 13913, 1115, 121, 105], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [104, 1503, 23421, 6407, 2768, 2254, 7593, 1124, 2768, 13913, 1145, 121, 105]}


In [34]:
#Define Evaluator
#!pip install rouge_score

In [35]:
from datasets import load_metric
rouge_metric = load_metric("rouge")

In [36]:
#Train Model
#data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding='longest', return_tensors='pt')
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding='longest', return_tensors='pt')


In [37]:
#model_name = 't5_gec_hindi_muRIL_1'
#torch_device = 'cuda' if torch.cuda.is_available() else 'cpu' 
#model = T5ForConditionalGeneration.from_pretrained(model_name).to(torch_device)

In [39]:
# defining training related arguments
batch_size = 8
'''
args = Seq2SeqTrainingArguments(output_dir="models/hindi/T5_muRIL_3",
                        evaluation_strategy="epoch",
                        per_device_train_batch_size=batch_size,
                        per_device_eval_batch_size=batch_size,
                        learning_rate=2e-5,
                        num_train_epochs=1,
                        weight_decay=0.1,
                        save_total_limit=2,
                        predict_with_generate=True,
                        fp16 = True,
                        gradient_accumulation_steps = 6,
                        #eval_steps = 500,
                        #save_steps = 2000,
                        #dataloader_num_workers= 4,
                        load_best_model_at_end=True,
                        logging_dir="logs/hindi/T5_muRIL_3")
'''

args = Seq2SeqTrainingArguments(
    output_dir="models/hindi/T5_muRIL_Full",
    #output_dir =None,
    evaluation_strategy="epoch",
    learning_rate=2e-3,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.3,
    save_steps=300000,
    num_train_epochs=10,
    dataloader_num_workers= 2,
    predict_with_generate=True,
    fp16=True,
    #logging_dir="logs/hindi/T5_muRIL_3"

)

In [40]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
#os.environ["CUDA_LAUNCH_BLOCKING"] = "false"
os.environ['TRANSFORMERS_NO_ADVISORY_WARNINGS'] = 'true'

In [41]:
import torch
#torch.cuda.empty_cache()

In [42]:
# defining trainer using 🤗
trainer = Seq2SeqTrainer(model=model, 
                args=args, 
                train_dataset= GrammarDataset(train_dataset, tokenizer),
                eval_dataset=GrammarDataset(test_dataset, tokenizer),
                tokenizer=tokenizer,
                data_collator=data_collator)
                #compute_metrics=compute_metrics)

Using cuda_amp half precision backend


In [43]:
#import torch
#torch.cuda.empty_cache()

In [44]:
import os
os.environ["WANDB_DISABLED"] = "true"     #Disabling Wandb
#model.resize_token_embeddings(len(tokenizer))
trainer.train()

***** Running training *****
  Num examples = 14288
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 24
  Gradient Accumulation steps = 1
  Total optimization steps = 5960
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss
1,2.464700,1.607950
2,2.404500,1.396081
3,2.327900,1.434922
4,2.210000,1.267827
5,2.144800,1.079509
6,1.744200,0.838862
7,1.432800,0.879704
8,1.320200,0.538478
9,0.999400,0.454238
10,0.851300,0.404103


***** Running Evaluation *****
  Num examples = 1429
  Batch size = 24
***** Running Evaluation *****
  Num examples = 1429
  Batch size = 24
***** Running Evaluation *****
  Num examples = 1429
  Batch size = 24
***** Running Evaluation *****
  Num examples = 1429
  Batch size = 24
***** Running Evaluation *****
  Num examples = 1429
  Batch size = 24
***** Running Evaluation *****
  Num examples = 1429
  Batch size = 24
***** Running Evaluation *****
  Num examples = 1429
  Batch size = 24
***** Running Evaluation *****
  Num examples = 1429
  Batch size = 24
***** Running Evaluation *****
  Num examples = 1429
  Batch size = 24
***** Running Evaluation *****
  Num examples = 1429
  Batch size = 24


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=5960, training_loss=1.7238809111934381, metrics={'train_runtime': 4660.5741, 'train_samples_per_second': 30.657, 'train_steps_per_second': 1.279, 'total_flos': 5281501299793920.0, 'train_loss': 1.7238809111934381, 'epoch': 10.0})

In [47]:
#Save the trained Model
trainer.save_model('t5_gec_hindi_muRIL_best_Full')

Saving model checkpoint to t5_gec_hindi_muRIL_best_Full
Configuration saved in t5_gec_hindi_muRIL_best_Full/config.json
Model weights saved in t5_gec_hindi_muRIL_best_Full/pytorch_model.bin
tokenizer config file saved in t5_gec_hindi_muRIL_best_Full/tokenizer_config.json
Special tokens file saved in t5_gec_hindi_muRIL_best_Full/special_tokens_map.json


In [45]:
import os
os.environ["WANDB_DISABLED"] = "true"     #Disabling Wandb
#model.resize_token_embeddings(len(tokenizer))
trainer.train()

***** Running training *****
  Num examples = 12859
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 24
  Gradient Accumulation steps = 1
  Total optimization steps = 1608


Epoch,Training Loss,Validation Loss
1,0.390700,0.628927
2,0.384600,0.628927
3,0.388100,0.628927


***** Running Evaluation *****
  Num examples = 1429
  Batch size = 24
***** Running Evaluation *****
  Num examples = 1429
  Batch size = 24
***** Running Evaluation *****
  Num examples = 1429
  Batch size = 24


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1608, training_loss=0.3878441734693537, metrics={'train_runtime': 1267.4907, 'train_samples_per_second': 30.436, 'train_steps_per_second': 1.269, 'total_flos': 1427035726725120.0, 'train_loss': 0.3878441734693537, 'epoch': 3.0})

In [46]:
#Save the trained Model
trainer.save_model('t5_gec_hindi_muRIL_best_Full')

Saving model checkpoint to t5_gec_hindi_muRIL_best_Full
Configuration saved in t5_gec_hindi_muRIL_best_Full/config.json
Model weights saved in t5_gec_hindi_muRIL_best_Full/pytorch_model.bin
tokenizer config file saved in t5_gec_hindi_muRIL_best_Full/tokenizer_config.json
Special tokens file saved in t5_gec_hindi_muRIL_best_Full/special_tokens_map.json


In [39]:
import os
os.environ["WANDB_DISABLED"] = "true"     #Disabling Wandb
#model.resize_token_embeddings(len(tokenizer))
trainer.train()

***** Running training *****
  Num examples = 2333596
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 24
  Gradient Accumulation steps = 1
  Total optimization steps = 97234
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss
1,0.113900,0.088758


Saving model checkpoint to models/hindi/T5_muRIL_Full/checkpoint-30000
Configuration saved in models/hindi/T5_muRIL_Full/checkpoint-30000/config.json
Model weights saved in models/hindi/T5_muRIL_Full/checkpoint-30000/pytorch_model.bin
tokenizer config file saved in models/hindi/T5_muRIL_Full/checkpoint-30000/tokenizer_config.json
Special tokens file saved in models/hindi/T5_muRIL_Full/checkpoint-30000/special_tokens_map.json
Saving model checkpoint to models/hindi/T5_muRIL_Full/checkpoint-60000
Configuration saved in models/hindi/T5_muRIL_Full/checkpoint-60000/config.json
Model weights saved in models/hindi/T5_muRIL_Full/checkpoint-60000/pytorch_model.bin
tokenizer config file saved in models/hindi/T5_muRIL_Full/checkpoint-60000/tokenizer_config.json
Special tokens file saved in models/hindi/T5_muRIL_Full/checkpoint-60000/special_tokens_map.json
Saving model checkpoint to models/hindi/T5_muRIL_Full/checkpoint-90000
Configuration saved in models/hindi/T5_muRIL_Full/checkpoint-90000/conf

TrainOutput(global_step=97234, training_loss=2.108593665541063, metrics={'train_runtime': 77686.3981, 'train_samples_per_second': 30.039, 'train_steps_per_second': 1.252, 'total_flos': 9.332473302411264e+16, 'train_loss': 2.108593665541063, 'epoch': 1.0})

In [38]:
import os
os.environ["WANDB_DISABLED"] = "true"     #Disabling Wandb
#model.resize_token_embeddings(len(tokenizer))
trainer.train()

***** Running training *****
  Num examples = 2333596
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 24
  Gradient Accumulation steps = 1
  Total optimization steps = 97234
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss
1,0.070400,0.059936


***** Running Evaluation *****
  Num examples = 259289
  Batch size = 24


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=97234, training_loss=0.7156300433809912, metrics={'train_runtime': 77398.8282, 'train_samples_per_second': 30.15, 'train_steps_per_second': 1.256, 'total_flos': 9.332804423264256e+16, 'train_loss': 0.7156300433809912, 'epoch': 1.0})

In [39]:
#Wandb API Key: bcdbdd5ee9d76a20c90b5f2a246eb45f14b341a9
import os
os.environ["WANDB_DISABLED"] = "true"     #Disabling Wandb
#model.resize_token_embeddings(len(tokenizer))
trainer.train()

***** Running training *****
  Num examples = 450000
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 24
  Gradient Accumulation steps = 1
  Total optimization steps = 18750
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss
1,0.132100,0.107768


***** Running Evaluation *****
  Num examples = 50000
  Batch size = 24


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=18750, training_loss=0.6001573170979818, metrics={'train_runtime': 14814.4253, 'train_samples_per_second': 30.376, 'train_steps_per_second': 1.266, 'total_flos': 1.799350678020096e+16, 'train_loss': 0.6001573170979818, 'epoch': 1.0})

In [40]:
#torch_device = 'cuda' if torch.cuda.is_available() else 'cpu' 

In [40]:
#Save the trained Model
trainer.save_model('t5_gec_hindi_muRIL_best')

Saving model checkpoint to t5_gec_hindi_muRIL_best
Configuration saved in t5_gec_hindi_muRIL_best/config.json
Model weights saved in t5_gec_hindi_muRIL_best/pytorch_model.bin
tokenizer config file saved in t5_gec_hindi_muRIL_best/tokenizer_config.json
Special tokens file saved in t5_gec_hindi_muRIL_best/special_tokens_map.json


In [41]:
#Save the trained Model
trainer.save_model('t5_gec_hindi_muRIL_best')

Saving model checkpoint to t5_gec_hindi_muRIL_best
Configuration saved in t5_gec_hindi_muRIL_best/config.json
Model weights saved in t5_gec_hindi_muRIL_best/pytorch_model.bin
tokenizer config file saved in t5_gec_hindi_muRIL_best/tokenizer_config.json
Special tokens file saved in t5_gec_hindi_muRIL_best/special_tokens_map.json


In [ ]:
#!zip -r 't5_gec_hindi_muRIL_2.zip' 't5_gec_hindi_muRIL_2'

In [42]:
print("Trained")

Trained


# Testing Trained Model

In [48]:
#Testing
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration,AutoTokenizer
#model_name = 't5_gec_hindi_muRIL'
#model_name = 't5_gec_hindi_muRIL_best'
model_name = 't5_gec_hindi_muRIL_best_Full'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu' 

tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")


loading configuration file config.json from cache at /DATA/gupta92/.cache/huggingface/hub/models--google--muril-base-cased/snapshots/afd9f36c7923d54e97903922ff1b260d091d202f/config.json
Model config BertConfig {
  "_name_or_path": "google/muril-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.22.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 197285
}

loading file vocab.txt from cache at /DATA/gupta92/.cache/huggingface/hub/models--google--muril-base-cased/snapshots/afd9f36c7923d54e97903922ff1b260d091d202

In [49]:
#tokenizer = AutoTokenizer.from_pretrained("surajp/RoBERTa-hindi-guj-san")
#tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")
trained_model = T5ForConditionalGeneration.from_pretrained(model_name).to(torch_device)

loading configuration file t5_gec_hindi_muRIL_best_Full/config.json
Model config T5Config {
  "_name_or_path": "t5_gec_hindi_muRIL_best_Full",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translatio

In [50]:
#trained_model

In [50]:
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        param = parameter.numel()
        table.add_row([name, param])
        total_params+=param
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

#count_parameters(trained_model)

In [51]:
param_size = 0
for param in trained_model.parameters():
    param_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in trained_model.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (param_size + buffer_size) / 1024**2
print('model size: {:.3f}MB'.format(size_all_mb))

model size: 1334.168MB


In [51]:
def correct_grammar(input_text,num_return_sequences):
    #batch = tokenizer([input_text],truncation=True,padding='max_length',max_length=64, return_tensors="pt").to(torch_device)
    batch = tokenizer([input_text],truncation=True,padding='max_length',max_length=64, return_tensors="pt",return_token_type_ids=False).to(device)
    #print(batch)
    #translated = model1.generate(**batch,max_length=64,num_beams=4, num_return_sequences=num_return_sequences, temperature=1.5)
    max_len =  train_df.input.str.len().max()
    #translated= trained_model.generate(**batch,max_length=64,num_beams=4, num_return_sequences=num_return_sequences, temperature=1.5)
    translated= trained_model.generate(**batch,max_length=max_len,num_beams=4, num_return_sequences=num_return_sequences, temperature=1.5)
    
    
    #print("here")
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    return tgt_text

In [47]:
#print(T5ForConditionalGeneration.__version__)

In [55]:
text = test_df['input'].iat[200]
correct = test_df['output'].iat[200]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
l = len(correct)
predicted_s = correct_grammar(text, num_return_sequences=4)[0]
p = len(predicted_s)

if(p>l):
    predicted_s = predicted_s[:l]
elif (p<l-4):
    predicted_s = text
print("Predicted Sentence:",predicted_s)

Incorrect Sentence: नागरमोथा की जड़ में कसेरू की भांति एक कन्द निकलता है जिसे नागरमोथा कहते हैं.
Correct Seentence: नागरमोथा की जड़ में कसेरू की भांति एक कन्द निकलता हैं जिसे नागरमोथा कहते हैं.
Predicted Sentence: नागरमोथा के जड़ में कसेरू की भांति एक कन्द निकलता है जिसे नागरमोथा कहते हैं. 


In [48]:
text = test_df['input'].iat[0]
correct = test_df['output'].iat[0]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
l = len(correct)
predicted_s = correct_grammar(text, num_return_sequences=4)[0]
p = len(predicted_s)

if(p>l):
    predicted_s = predicted_s[:l]
elif (p<l-4):
    predicted_s = text
print("Predicted Sentence:",predicted_s)

Incorrect Sentence: यह शहर मलक्का जलसन्धि पर बसा हुआ हैं.
Correct Seentence: यह शहर मलक्का जलसन्धि पर बसा हुआ है.
Predicted Sentence: यह शहर मलक्का जलसन्धि पर बसा हुआ है.


In [46]:
text = test_df['input'].iat[0]
correct = test_df['output'].iat[0]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
l = len(correct)
predicted_s = correct_grammar(text, num_return_sequences=2)[0]
p = len(predicted_s)

if(p>l):
    predicted_s = predicted_s[:l]
elif (p<l-4):
    predicted_s = text


print("Predicted Sentence:",predicted_s)

Incorrect Sentence: दरभंगा, बिहार के एक प्रखण्ड.
Correct Seentence: दरभंगा, बिहार का एक प्रखण्ड.
Predicted Sentence: दरभंगा, बिहार का एक प्रखण्ड.


In [50]:
text = test_df['input'].iat[100]
correct = test_df['output'].iat[100]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
l = len(correct)
predicted_s = correct_grammar(text, num_return_sequences=2)[0]
p = len(predicted_s)

if(p>l):
    predicted_s = predicted_s[:l]
elif (p<l-4):
    predicted_s = text


print("Predicted Sentence:",predicted_s)

Incorrect Sentence: अन् ‍ त पुर को लै गईं, जहां न दूजो जाय.
Correct Seentence: अन् ‍ त पुर को लै गए, जहां न दूजो जाय.
Predicted Sentence: अन् त पुर को लै गया, जहां न दूजो जाय. 


In [51]:
text = test_df['input'].iat[43]
correct = test_df['output'].iat[43]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
p = len(predicted_s)
l = len(correct)
predicted_s = correct_grammar(text, num_return_sequences=2)[0]
p = len(predicted_s)
if(p>l):
    predicted_s = predicted_s[:l]

print("Predicted Sentence:",predicted_s)

Incorrect Sentence: अंतर केवल इतना ही है कि थोड़े स्थान में सभी सुविधाएं प्रदान करने का प्रयास कीं जाता है.
Correct Seentence: अंतर केवल इतना ही है कि थोड़े स्थान में सभी सुविधाएं प्रदान करने का प्रयास किया जाता है.
Predicted Sentence: अंतर केवल इतना ही है कि थोड़े स्थान में सभी सुविधाएं प्रदान करने का प्रयास किया जाता है.


In [49]:
text = test_df['input'].iat[1]
correct = test_df['output'].iat[1]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
p = len(predicted_s)
l = len(correct)
predicted_s = correct_grammar(text, num_return_sequences=4)[0][:l]
print("Predicted Sentence:",predicted_s)

Incorrect Sentence: दिल्ली का गुर्जरों ने मालगुजारी बहादुरशाह जफर मुगल बादशाह को देनी शुरू कर दी थी.
Correct Seentence: दिल्ली के गुर्जरों ने मालगुजारी बहादुरशाह जफर मुगल बादशाह को देनी शुरू कर दी थी.
Predicted Sentence: दिल्ली के गुर्जरों ने मालगुजारी बहादुरशाह जफर मुगल बादशाह को देना शुरू कर दी थी.


In [54]:
text = test_df['input'].iat[50]
correct = test_df['output'].iat[50]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
predicted_s = correct_grammar(text, num_return_sequences=1)[0]
l = len(correct)
p = len(predicted_s)
if(p>l):
    predicted_s = predicted_s[:l]

print("Predicted Sentence:",predicted_s)

Incorrect Sentence: वे ऐसी पहली राष्ट्रदेवी हैं, जिनकी मातृभाषा अंग्रेज़ी नहीं है.
Correct Seentence: वे ऐसी पहली राष्ट्रदेवी है, जिनकी मातृभाषा अंग्रेज़ी नहीं है.
Predicted Sentence: वे ऐसी पहली राष्ट्रदेवी हैं, जिनकी मातृभाषा अंग्रेज़ी नहीं है


In [55]:
text = test_df['input'].iat[60]
correct = test_df['output'].iat[60]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
predicted_s = correct_grammar(text, num_return_sequences=1)[0]
l = len(correct)
p = len(predicted_s)
if(p>l):
    predicted_s = predicted_s[:l]
print("Predicted Sentence:",predicted_s)


Incorrect Sentence: मराक गुरसुता  10 दिसंबर 1945 09 अक्टूबर 2006  पोलैंडी गायक, संगीतकार, कवि और चित्रकार था.
Correct Seentence: मराक गुरसुता  10 दिसंबर 1945 09 अक्टूबर 2006  पोलैंडी गायक, संगीतकार, कवि और चित्रकार हैं.
Predicted Sentence: मराक गुरसुता 10 दिसंबर 1945 09 अक्टूबर 2006 पोलैंडी गायक, संगीतकार, कवि और चित्रकार थे.


In [56]:
text = test_df['input'].iat[51]
correct = test_df['output'].iat[51]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
predicted_s = correct_grammar(text, num_return_sequences=1)[0]
print("Predicted Sentence:",predicted_s)

Incorrect Sentence: आज हम आपको हार्दिक पांड्या की ऐसी 5 बाते बताने जा रहे है जिन्हें आप शायद ही जानते होंगे.
Correct Seentence: आज हम आपको हार्दिक पांड्या की ऐसी 5 बाते बताने जा रहे हैं जिन्हें आप शायद ही जानते होंगे.
Predicted Sentence: आज हम आपको हार्दिक पांड्या की ऐसी 5 बाते बताने जा रहे हैं जिन्हें आप शायद ही जानते होंगे.


In [68]:
text = test_df['input'].iat[20]
correct = test_df['output'].iat[20]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
predicted_s = correct_grammar(text, num_return_sequences=1)[0]
p=len(predicted_s)
l = len(text)
if(p>l):
    predicted_s = predicted_s[:l]
print("Predicted Sentence:",predicted_s)

Incorrect Sentence: प्रारंभिक जीवन और परिवार के इतिहास.
Correct Seentence: प्रारंभिक जीवन और परिवार का इतिहास.
Predicted Sentence: प्रारंभिक जीवन और परिवार की इतिहास.


# Performance Analysis
1. BLEU Score
2. GLEU Score


In [50]:
test_df.shape

(259289, 3)

In [51]:
test_df.head(5)

,input,output,input_token_len
677358,"अब किशन अपने भाई का बदला लेने की सोचता है, पर कई ओर से घिरी भी होता है.","अब किशन अपने भाई का बदला लेने की सोचता है, पर कई ओर से घिरा भी होता है.",23
2028550,इसे इल्देजिज़ के दोनों पुत्रों मुहम्मद और किज़िल अरसतन को समर्पित कीं गया था.,इसे इल्देजिज़ के दोनों पुत्रों मुहम्मद और किज़िल अरसतन को समर्पित किया गया था.,24
328281,शव को डेढ़ लाख लोगों ने जुलूस निकाल कर पूरे शहर में घुमाते हुए राप्ती नदी के किनारे राजघाट पर उसका अन्तिम संस्कार कर दी.,शव को डेढ़ लाख लोगों ने जुलूस निकाल कर पूरे शहर में घुमाते हुए राप्ती नदी के किनारे राजघाट पर उसका अन्तिम संस्कार कर दिया.,31
188804,"सूखी महीनों या वर्षों तक रह सकता है, और इसे 15 दिनों के बाद घोषित किया जा सकता है.","सूखा महीनों या वर्षों तक रह सकता है, और इसे 15 दिनों के बाद घोषित किया जा सकता है.",23
2513989,भांडुप मुंबई की एक उपनगर है.,भांडुप मुंबई का एक उपनगर है.,11


In [52]:
test_df

,input,output,input_token_len
677358,"अब किशन अपने भाई का बदला लेने की सोचता है, पर कई ओर से घिरी भी होता है.","अब किशन अपने भाई का बदला लेने की सोचता है, पर कई ओर से घिरा भी होता है.",23
2028550,इसे इल्देजिज़ के दोनों पुत्रों मुहम्मद और किज़िल अरसतन को समर्पित कीं गया था.,इसे इल्देजिज़ के दोनों पुत्रों मुहम्मद और किज़िल अरसतन को समर्पित किया गया था.,24
328281,शव को डेढ़ लाख लोगों ने जुलूस निकाल कर पूरे शहर में घुमाते हुए राप्ती नदी के किनारे राजघाट पर उसका अन्तिम संस्कार कर दी.,शव को डेढ़ लाख लोगों ने जुलूस निकाल कर पूरे शहर में घुमाते हुए राप्ती नदी के किनारे राजघाट पर उसका अन्तिम संस्कार कर दिया.,31
188804,"सूखी महीनों या वर्षों तक रह सकता है, और इसे 15 दिनों के बाद घोषित किया जा सकता है.","सूखा महीनों या वर्षों तक रह सकता है, और इसे 15 दिनों के बाद घोषित किया जा सकता है.",23
2513989,भांडुप मुंबई की एक उपनगर है.,भांडुप मुंबई का एक उपनगर है.,11
...,...,...,...
2405409,आचार्य धर्मपाल नालंदा महाविहार के कुलपति थे जिनका शिष्य शीलभद्र ह्वेन्त्सांग को महायान के प्रमुख ग्रंथों का अध्यापन कराया था.,आचार्य धर्मपाल नालंदा महाविहार के कुलपति थे जिनके शिष्य शीलभद्र ह्वेन्त्सांग को महायान के प्रमुख ग्रंथों का अध्यापन कराया था.,30
985351,औरतें तीज और करवी चौथ की जगह स्वतंत्रता दिवस और गणतंत्रता दिवस का आनंद लेगी.,औरतें तीज और करवा चौथ की जगह स्वतंत्रता दिवस और गणतंत्रता दिवस का आनंद लेगी.,20
1911740,एकदरबेला नेपाल के जनकपुर अंचल का महोत्तरी जिला के एक गांव विकास समिति है.,एकदरबेला नेपाल के जनकपुर अंचल का महोत्तरी जिला का एक गांव विकास समिति है.,19
621202,यहां रहने वाले कई जीवों में जीवदीप्ति भी देखी जाती हैं.,यहां रहने वाले कई जीवों में जीवदीप्ति भी देखी जाती है.,16


In [54]:
from nltk.translate.gleu_score import sentence_gleu
GLEU_val_emb = []
test_data = test_df.head(100)
print(test_data.shape)
#print("here")
itr = 0
for ind,i in tqdm(test_data.iterrows(),position=0):
    try:
        #print(ind,i)
        itr+=1
        text = str(i.input)
        #print(text)
        #pred = predict(str(i.enc_input),model)[0].split()
        pred = correct_grammar(text, num_return_sequences=1)[0]
        l = len(text)
        p = len(pred)
        if(p>l):
            pred = pred[:l]
            
        act = [str(i.output).split()]
        pred_s = str(pred).split()
        b = sentence_gleu(act,pred_s)
        GLEU_val_emb.append(b)
        if(itr%100 ==0):
            print("GELU Score for ",itr," data point",np.mean(GLEU_val_emb))
    except:
        print("Error") 
        continue
print("GELU Score = ",np.mean(GLEU_val_emb))

(100, 3)


2it [00:04,  2.34s/it]

Error


100it [04:22,  2.63s/it]

GELU Score for  100  data point 0.9157751050731273
GELU Score =  0.9157751050731273


In [51]:
from nltk.translate.gleu_score import sentence_gleu
GLEU_val_emb = []
test_data = test_df.head(100)
print(test_data.shape)
#print("here")
itr = 0
for ind,i in tqdm(test_data.iterrows(),position=0):
    try:
        #print(ind,i)
        itr+=1
        text = str(i.input)
        #print(text)
        #pred = predict(str(i.enc_input),model)[0].split()
        pred = correct_grammar(text, num_return_sequences=1)[0]
        l = len(text)
        p = len(pred)
        if(p>l):
            pred = pred[:l]
            
        act = [str(i.output).split()]
        pred_s = str(pred).split()
        b = sentence_gleu(act,pred_s)
        GLEU_val_emb.append(b)
        if(itr%100 ==0):
            print("GELU Score for ",itr," data point",np.mean(GLEU_val_emb))
    except:
        print("Error") 
        continue
print("GELU Score = ",np.mean(GLEU_val_emb))

(1000, 3)


100it [02:39,  1.62s/it]

GELU Score for  100  data point 0.9369473923868874


200it [05:21,  1.67s/it]

GELU Score for  200  data point 0.9203796555842938


...

GELU Score for  800  data point 0.9100877590947873


900it [24:12,  1.76s/it]

GELU Score for  900  data point 0.9116741504812822


1000it [26:55,  1.62s/it]

GELU Score for  1000  data point 0.9137746486265554
GELU Score =  0.9137746486265554


# Evaluation on Etoori's Test Dataset

In [9]:
import pandas as pd
df_test = pd.read_csv('DATA/etoori_test.csv')
df_test.shape

(30000, 2)

In [10]:
df_test.head(5)

,enc_input,dec_input
0,इसके अलावा माइकल शूमाकर द्वारा चलाई गई एक फरार...,इसके अलावा माइकल शूमाकर द्वारा चलाई गई एक फरार...
1,यह मन को काबू में करने वाली मुद्रा हैं इसीलिए ...,यह मन को काबू में करने वाली मुद्रा है इसीलिए इ...
2,आप पुस्तक पढ़ने में तल्लीन हैं और बच्चा उसनेाँ...,आप पुस्तक पढ़ने में तल्लीन हैं और बच्चा वहाँ प...
3,परिवार के मुताबिक धमाकों में हिस्सा लेने वाले ...,परिवार के मुताबिक धमाकों में हिस्सा लेने वाले ...
4,उनकी वो वाली बात भी अनिश्चित रहती हैं ।,उनकी वो वाली बात भी अनिश्चित रहती है ।


In [13]:
text = df_test['enc_input'].iat[0]
correct = df_test['dec_input'].iat[0]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
predicted_s = correct_grammar(text, num_return_sequences=1)[0]
l = len(correct)
p = len(predicted_s)
if(p>l):
    predicted_s = predicted_s[:l]

print("Predicted Sentence:",predicted_s)

Incorrect Sentence: इसके अलावा माइकल शूमाकर द्वारा चलाई गई एक फरारी कार भी को मिलियन डॉलर क्लब में माना जा रहा है ।
Correct Seentence: इसके अलावा माइकल शूमाकर द्वारा चलाई गई एक फरारी कार को भी मिलियन डॉलर क्लब में माना जा रहा है । 
Predicted Sentence: इसके अलावा माइकल शूमाकर द्वारा चलाई गई एक फरारी कार को भी मिलियन डॉलर क्लब में माना जा रहा है ।


In [17]:
text = df_test['enc_input'].iat[50]
correct = df_test['dec_input'].iat[50]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
predicted_s = correct_grammar(text, num_return_sequences=1)[0]
l = len(correct)
p = len(predicted_s)
if(p>l):
    predicted_s = predicted_s[:l]

print("Predicted Sentence:",predicted_s)

Incorrect Sentence: घर में घुस के बाद भी सुकून नहीं ।
Correct Seentence: घर में घुसने के बाद भी सुकून नहीं । 
Predicted Sentence: घर में घुसने के बाद भी सुकून नहीं ।


In [14]:
%%time
predicted_s = correct_grammar(text, num_return_sequences=1)[0]

CPU times: user 1.26 s, sys: 84.4 ms, total: 1.34 s
Wall time: 1.35 s


In [59]:
text = df_test['enc_input'].iat[1]
correct = df_test['dec_input'].iat[1]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
predicted_s = correct_grammar(text, num_return_sequences=1)[0]
l = len(correct)
p = len(predicted_s)
if(p>l):
    predicted_s = predicted_s[:l]
print("Predicted Sentence:",predicted_s)

Incorrect Sentence: यह मन को काबू में करने वाली मुद्रा हैं इसीलिए इसे चित्त हस्त मुद्रा योग कहते हैंं ।
Correct Seentence: यह मन को काबू में करने वाली मुद्रा है इसीलिए इसे चित्त हस्त मुद्रा योग कहते हैं । 
Predicted Sentence: यह मन को काबू में करने वाली मुद्रा है इसीलिए इसे चित्त हस्त मुद्रा योग कहते हैं ।


In [60]:
text = df_test['enc_input'].iat[2]
correct = df_test['dec_input'].iat[2]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
predicted_s = correct_grammar(text, num_return_sequences=1)[0]
l = len(correct)
p = len(predicted_s)
if(p>l):
    predicted_s = predicted_s[:l]
print("Predicted Sentence:",predicted_s)

Incorrect Sentence: आप पुस्तक पढ़ने में तल्लीन हैं और बच्चा उसनेाँ पर शोर मचा रहा है ।
Correct Seentence: आप पुस्तक पढ़ने में तल्लीन हैं और बच्चा वहाँ पर शोर मचा रहा है । 
Predicted Sentence: आप पुस्तक पढ़ने में तल्लीन हैं और बच्चा वहाँ पर शोर मचा रहा है ।


In [61]:
text = df_test['enc_input'].iat[3]
correct = df_test['dec_input'].iat[3]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
predicted_s = correct_grammar(text, num_return_sequences=1)[0]
l = len(correct)
p = len(predicted_s)
if(p>l):
    predicted_s = predicted_s[:l]
print("Predicted Sentence:",predicted_s)

Incorrect Sentence: परिवार के मुताबिक धमाकों में हिस्सा लेने वाले कोई दूसरे लोग थे और ये गलत पहचान का मामला हैं ।
Correct Seentence: परिवार के मुताबिक धमाकों में हिस्सा लेने वाले कोई दूसरे लोग थे और ये गलत पहचान का मामला है । 
Predicted Sentence: परिवार के मुताबिक धमाकों में हिस्सा लेने वाले कोई दूसरे लोग थे और ये गलत पहचान का मामला है ।


In [62]:
text = df_test['enc_input'].iat[4]
correct = df_test['dec_input'].iat[4]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
predicted_s = correct_grammar(text, num_return_sequences=1)[0]
l = len(correct)
p = len(predicted_s)
if(p>l):
    predicted_s = predicted_s[:l]
print("Predicted Sentence:",predicted_s)

Incorrect Sentence: उनकी वो वाली बात भी अनिश्चित रहती हैं ।
Correct Seentence: उनकी वो वाली बात भी अनिश्चित रहती है । 
Predicted Sentence: उनकी वो वाली बात भी अनिश्चित रहती है ।


In [64]:
text = df_test['enc_input'].iat[9]
correct = df_test['dec_input'].iat[9]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
predicted_s = correct_grammar(text, num_return_sequences=1)[0]
l = len(correct)
p = len(predicted_s)
if(p>l):
    predicted_s = predicted_s[:l]
print("Predicted Sentence:",predicted_s)

Incorrect Sentence: मैंने ललित हूँ ।
Correct Seentence: मैं ललित हूँ । 
Predicted Sentence: मैं ललित हूँ । 


In [ ]:
import nltk.translate.bleu_score as bleu
BLEU = []
index = []
test_data = df_test.head(10000)
np.random.seed(1)
for ind,i in tqdm(test_data.iterrows(),position=0):
    try:
        text = str(i.enc_input)
        #pred = predict(str(i.enc_input),model)[0].split()
        pred = correct_grammar(text, num_return_sequences=1)[0]
        act = [str(i.dec_input).split()]
        pred_s = str(pred).split()
        #print(act)
        #print(pred_s)
        b = bleu.sentence_bleu(act,pred_s)
        BLEU.append(b)
    except:
        index.append(ind)
        continue
print("BELU = ", np.mean(BLEU))

In [64]:
from nltk.translate.gleu_score import sentence_gleu
GLEU_val_emb = []
test_data = df_test.head(10000)
print(test_data.shape)
for ind,i in tqdm(test_data.iterrows(),position=0):
    try:
        text = str(i.enc_input)
        #pred = predict(str(i.enc_input),model)[0].split()
        pred = correct_grammar(text, num_return_sequences=1)[0]
        l = len(text)
        p = len(pred)
        if(p>l):
            pred = pred[:l]
            
        act = [str(i.dec_input).split()]
        pred_s = str(pred).split()
        b = sentence_gleu(act,pred_s)
        GLEU_val_emb.append(b)
        if(ind%1000 ==0):
            print("GELU Score for ",ind+1," data point",np.mean(GLEU_val_emb))
    except:
        continue
print("GELU Score = ",np.mean(GLEU_val_emb))

(10000, 2)


1it [00:01,  1.41s/it]

GELU Score for  1  data point 1.0


1001it [21:58,  1.20s/it]

GELU Score for  1001  data point 0.9150283140478553


...

GELU Score for  8001  data point 0.9201028812376454


9001it [2:53:42,  1.13s/it]

GELU Score for  9001  data point 0.9205000001846352


10000it [3:12:33,  1.16s/it]

GELU Score =  0.9208625332430048


In [65]:
print("GELU Score = ",np.mean(GLEU_val_emb))

GELU Score =  0.9208625332430048


# Testing HiWikEd test dataset

In [56]:
import pandas as pd
df_test1 = pd.read_csv('DATA/HiWikEd.csv')
df_test1.shape
df_test1 = df_test1.drop(['Unnamed: 0'], axis=1)

In [57]:
df_test1.head(5)

,enc_input,dec_input
0,चाय की दुकान से लेकर वाहनों और दिवारों तक हर जगह विज्ञापन ही विज्ञापन दिखाई देते है.,चाय की दुकान से लेकर वाहनों और दिवारों तक हर जगह विज्ञापन ही विज्ञापन दिखाई देते हैं.
1,"ये कहीं पे निगाहें, कही पे निशाना का सा अन्दाज है.","यह कहीं पे निगाहें, कही पे निशाना का सा अन्दाज है."
2,आज हम विज्ञापन युग के सीमान्त पर आ खड़े हुए है.,आज हम विज्ञापन युग के सीमान्त पर आ खड़े हुए हैं.
3,घर के बाहर पैर रखते ही हम विज्ञापन की दुनिया में घिर जाते है.,घर के बाहर पैर रखते ही हम विज्ञापन की दुनिया से घिर जाते है.
4,वर्तमान समय में विज्ञापन के कोई रूप हमारे सामने आते है.,वर्तमान समय में विज्ञापन के कई रूप हमारे सामने आते है.


In [12]:
text = df_test1['enc_input'].iat[45]
correct = df_test1['dec_input'].iat[45]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
predicted_s = correct_grammar(text, num_return_sequences=1)[0]
l = len(correct)
p = len(predicted_s)
if(p>l):
    predicted_s = predicted_s[:l]

print("Predicted Sentence:",predicted_s)

Incorrect Sentence: बादशाह के समक्ष एक लड़ाई में सिर काटे जाने पर भी उसका धड़ बहुत समय तक लड़ता रहा था.
Correct Seentence: बादशाह के समक्ष एक लड़ाई में सिर काटे जाने पर भी उनका धड़ बहुत समय तक लड़ता रहा था.
Predicted Sentence: बादशाह के समक्ष एक लड़ाई में सिर काटे जाने पर भी उसका धड़ बहुत समय तक लड़ता रहा था.


In [13]:
text = df_test1['enc_input'].iat[40]
correct = df_test1['dec_input'].iat[40]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
predicted_s = correct_grammar(text, num_return_sequences=1)[0]
l = len(correct)
p = len(predicted_s)
if(p>l):
    predicted_s = predicted_s[:l]

print("Predicted Sentence:",predicted_s)

Incorrect Sentence: इसे गौरी शंकर मंदिर भी कहते है.
Correct Seentence: इसे गौरी शंकर मंदिर भी कहते हैं.
Predicted Sentence: इसे गौरी शंकर मंदिर भी कहते हैं.


In [62]:
import warnings 
warnings.filterwarnings('ignore')
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm
import numpy as np
from nltk.translate.gleu_score import sentence_gleu


GLEU_val_emb = []
test_data = df_test1.head(100)
print(test_data.shape)

for ind,i in tqdm(test_data.iterrows(),position=0):
    try:
        text = str(i.enc_input)
        #pred = predict(str(i.enc_input),model)[0].split()
        pred = correct_grammar(text, num_return_sequences=1)[0]
        l = len(text)
        p = len(pred)
        if(p>l):
            pred = pred[:l]
            
        act = [str(i.dec_input).split()]
        pred_s = str(pred).split()
        b = sentence_gleu(act,pred_s)
        GLEU_val_emb.append(b)
        if(ind%100 ==0):
            print("GELU Score for ",ind+1," data point",np.mean(GLEU_val_emb))
    except:
        continue
print("GELU Score = ",np.mean(GLEU_val_emb))

(100, 2)


1it [00:02,  2.80s/it]

GELU Score for  1  data point 0.9354838709677419


100it [04:20,  2.60s/it]

GELU Score =  0.8107857635662435


In [52]:
import pandas as pd
df_test_pron_infl = pd.read_csv('DATA/Wikiedits_PRON_INFL.csv')
print("PRON_INFL:", df_test_pron_infl.shape)

df_test_adj_infl = pd.read_csv('DATA/Wikiedits_ADJ_INFL.csv')
print("ADJ_INFL:", df_test_adj_infl.shape)

df_test_adp_infl = pd.read_csv('DATA/Wikiedits_ADP_INFL.csv')
print("ADP_INFL:", df_test_adp_infl.shape)

df_test_verb_infl = pd.read_csv('DATA/Wikiedits_VERB_INFL.csv')
print("VERB_INFL:", df_test_verb_infl.shape)

df_test_noun_infl = pd.read_csv('DATA/Wikiedits_NOUN_INFL.csv')
print("NOUN_INFL:", df_test_noun_infl.shape)

PRON_INFL: (496, 2)
ADJ_INFL: (150, 2)
ADP_INFL: (1101, 2)
VERB_INFL: (5241, 2)
NOUN_INFL: (182, 2)


In [15]:
df_test_pron_infl.head(2)

,enc_input,dec_input
0,नैबरिजा ने अपनी भाषा पठन अध्ययन को चार पुस्तको...,नैबरिजा ने अपने भाषा पठन अध्ययन को चार पुस्तको...
1,अपने मूल प्राकृतिक स्थिति में ये उत्तर अमेरिका...,अपनी मूल प्राकृतिक स्थिति में ये उत्तर अमेरिका...


In [17]:
df_test_adj_infl.head(2)

,enc_input,dec_input
0,पहली समान विद्यालय व्यवस्था और दूसरी देश की शि...,पहली समान विद्यालय व्यवस्था और दूसरे देश की शि...
1,नोएडा एशिया की सबसे बड़ी औद्योगिक उपनगरों में ...,नोएडा एशिया के सबसे बड़े औद्योगिक उपनगरों में ...


In [18]:
import warnings 
warnings.filterwarnings('ignore')
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm
import numpy as np
from nltk.translate.gleu_score import sentence_gleu


GLEU_val_emb = []
test_data = df_test_adj_infl
print(test_data.shape)

for ind,i in tqdm(test_data.iterrows(),position=0):
    try:
        text = str(i.enc_input)
        #pred = predict(str(i.enc_input),model)[0].split()
        pred = correct_grammar(text, num_return_sequences=1)[0]
        l = len(text)
        p = len(pred)
        if(p>l):
            pred = pred[:l]
            
        act = [str(i.dec_input).split()]
        pred_s = str(pred).split()
        b = sentence_gleu(act,pred_s)
        GLEU_val_emb.append(b)
        if(ind%50 ==0):
            print("GELU Score for ",ind+1," data point",np.mean(GLEU_val_emb))
    except:
        continue
print("GELU Score = ",np.mean(GLEU_val_emb))

(150, 2)


1it [00:01,  1.72s/it]

GELU Score for  1  data point 0.8484848484848485


51it [01:13,  1.54s/it]

GELU Score for  51  data point 0.7370891501571076


101it [02:19,  1.31s/it]

GELU Score for  101  data point 0.7305455277794598


150it [03:24,  1.37s/it]

GELU Score =  0.7434265640344555


In [19]:
print("GELU Score for ADJ_INFL Error= ",np.mean(GLEU_val_emb))

GELU Score for ADJ_INFL Error=  0.7434265640344555


In [20]:
import warnings 
warnings.filterwarnings('ignore')
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm
import numpy as np
from nltk.translate.gleu_score import sentence_gleu


GLEU_val_emb = []
test_data = df_test_pron_infl
print(test_data.shape)

for ind,i in tqdm(test_data.iterrows(),position=0):
    try:
        text = str(i.enc_input)
        #pred = predict(str(i.enc_input),model)[0].split()
        pred = correct_grammar(text, num_return_sequences=1)[0]
        l = len(text)
        p = len(pred)
        if(p>l):
            pred = pred[:l]
            
        act = [str(i.dec_input).split()]
        pred_s = str(pred).split()
        b = sentence_gleu(act,pred_s)
        GLEU_val_emb.append(b)
        if(ind%100 ==0):
            print("GELU Score for ",ind+1," data point",np.mean(GLEU_val_emb))
    except:
        continue
print("GELU Score = ",np.mean(GLEU_val_emb))

(496, 2)


1it [00:01,  1.41s/it]

GELU Score for  1  data point 0.6578947368421053


101it [02:15,  1.30s/it]

GELU Score for  101  data point 0.8025024854959928


...

GELU Score for  301  data point 0.8094019378566462


401it [08:51,  1.41s/it]

GELU Score for  401  data point 0.8143655029809677


496it [10:57,  1.33s/it]

GELU Score =  0.8095978571769412


In [21]:
print("GELU Score for PRON_INFL Error= ",np.mean(GLEU_val_emb))

GELU Score for PRON_INFL Error=  0.8095978571769412


In [53]:
import warnings 
warnings.filterwarnings('ignore')
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm
import numpy as np
from nltk.translate.gleu_score import sentence_gleu


GLEU_val_emb = []
test_data = df_test_adp_infl.head(100)
print(test_data.shape)

for ind,i in tqdm(test_data.iterrows(),position=0):
    try:
        text = str(i.enc_input)
        #pred = predict(str(i.enc_input),model)[0].split()
        pred = correct_grammar(text, num_return_sequences=1)[0]
        l = len(text)
        p = len(pred)
        if(p>l):
            pred = pred[:l]
            
        act = [str(i.dec_input).split()]
        pred_s = str(pred).split()
        b = sentence_gleu(act,pred_s)
        GLEU_val_emb.append(b)
        if(ind%200 ==0):
            print("GELU Score for ",ind+1," data point",np.mean(GLEU_val_emb))
    except:
        continue
print("GELU Score = ",np.mean(GLEU_val_emb))

(100, 2)


1it [00:04,  4.00s/it]

GELU Score for  1  data point 0.8


100it [07:03,  4.24s/it]

GELU Score =  0.7324791038189717


In [54]:
print("GELU Score for ADP_INFL Error= ",np.mean(GLEU_val_emb))

GELU Score for ADP_INFL Error=  0.7324791038189717


In [22]:
import warnings 
warnings.filterwarnings('ignore')
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm
import numpy as np
from nltk.translate.gleu_score import sentence_gleu


GLEU_val_emb = []
test_data = df_test_adp_infl
print(test_data.shape)

for ind,i in tqdm(test_data.iterrows(),position=0):
    try:
        text = str(i.enc_input)
        #pred = predict(str(i.enc_input),model)[0].split()
        pred = correct_grammar(text, num_return_sequences=1)[0]
        l = len(text)
        p = len(pred)
        if(p>l):
            pred = pred[:l]
            
        act = [str(i.dec_input).split()]
        pred_s = str(pred).split()
        b = sentence_gleu(act,pred_s)
        GLEU_val_emb.append(b)
        if(ind%200 ==0):
            print("GELU Score for ",ind+1," data point",np.mean(GLEU_val_emb))
    except:
        continue
print("GELU Score = ",np.mean(GLEU_val_emb))

(1101, 2)


1it [00:01,  1.38s/it]

GELU Score for  1  data point 0.8


201it [04:24,  1.26s/it]

GELU Score for  201  data point 0.827990913010508


...

GELU Score for  801  data point 0.8080139789093645


1001it [21:57,  1.29s/it]

GELU Score for  1001  data point 0.8102707580849008


1101it [24:11,  1.32s/it]

GELU Score =  0.8082467684365293


In [23]:
print("GELU Score for ADP_INFL Error= ",np.mean(GLEU_val_emb))

GELU Score for ADP_INFL Error=  0.8082467684365293


In [24]:
import warnings 
warnings.filterwarnings('ignore')
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm
import numpy as np
from nltk.translate.gleu_score import sentence_gleu


GLEU_val_emb = []
test_data = df_test_verb_infl
print(test_data.shape)

for ind,i in tqdm(test_data.iterrows(),position=0):
    try:
        text = str(i.enc_input)
        #pred = predict(str(i.enc_input),model)[0].split()
        pred = correct_grammar(text, num_return_sequences=1)[0]
        l = len(text)
        p = len(pred)
        if(p>l):
            pred = pred[:l]
            
        act = [str(i.dec_input).split()]
        pred_s = str(pred).split()
        b = sentence_gleu(act,pred_s)
        GLEU_val_emb.append(b)
        if(ind%1000 ==0):
            print("GELU Score for ",ind+1," data point",np.mean(GLEU_val_emb))
    except:
        continue
print("GELU Score = ",np.mean(GLEU_val_emb))

(5241, 2)


1it [00:01,  1.28s/it]

GELU Score for  1  data point 0.9354838709677419


1001it [21:59,  1.27s/it]

GELU Score for  1001  data point 0.8282614573022717


...

GELU Score for  4001  data point 0.8287928854774717


5001it [1:49:50,  1.30s/it]

GELU Score for  5001  data point 0.8294709694830051


5241it [1:55:08,  1.32s/it]

GELU Score =  0.8298811959742303


In [25]:
print("GELU Score for VERB_INFL Error= ",np.mean(GLEU_val_emb))

GELU Score for VERB_INFL Error=  0.8298811959742303


In [53]:
import warnings 
warnings.filterwarnings('ignore')
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm
import numpy as np
from nltk.translate.gleu_score import sentence_gleu


GLEU_val_emb = []
test_data = df_test_noun_infl
print(test_data.shape)

for ind,i in tqdm(test_data.iterrows(),position=0):
    try:
        text = str(i.enc_input)
        #pred = predict(str(i.enc_input),model)[0].split()
        pred = correct_grammar(text, num_return_sequences=1)[0]
        l = len(text)
        p = len(pred)
        if(p>l):
            pred = pred[:l]
            
        act = [str(i.dec_input).split()]
        pred_s = str(pred).split()
        b = sentence_gleu(act,pred_s)
        GLEU_val_emb.append(b)
        if(ind%50 ==0):
            print("GELU Score for ",ind+1," data point",np.mean(GLEU_val_emb))
    except:
        continue
print("GELU Score = ",np.mean(GLEU_val_emb))

(182, 2)


1it [00:06,  6.56s/it]

GELU Score for  1  data point 0.8780487804878049


51it [05:44,  6.70s/it]

GELU Score for  51  data point 0.693378284921975


...

GELU Score for  101  data point 0.7262667914196586


151it [16:52,  6.86s/it]

GELU Score for  151  data point 0.7131101355914514


182it [20:14,  6.68s/it]

GELU Score =  0.7170992122074078


In [54]:
print("GELU Score for NOUN_INFL Error= ",np.mean(GLEU_val_emb))

GELU Score for NOUN_INFL Error=  0.7170992122074078


In [26]:
df_test1.shape

(13187, 2)

In [58]:
import warnings 
warnings.filterwarnings('ignore')
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm
import numpy as np
from nltk.translate.gleu_score import sentence_gleu


GLEU_val_emb = []
test_data = df_test1.head(100)
print(df_test1.shape)

for ind,i in tqdm(test_data.iterrows(),position=0):
    try:
        text = str(i.enc_input)
        #pred = predict(str(i.enc_input),model)[0].split()
        pred = correct_grammar(text, num_return_sequences=1)[0]
        l = len(text)
        p = len(pred)
        if(p>l):
            pred = pred[:l]
            
        act = [str(i.dec_input).split()]
        pred_s = str(pred).split()
        b = sentence_gleu(act,pred_s)
        GLEU_val_emb.append(b)
        if(ind%20 ==0):
            print("GELU Score for ",ind+1," data point",np.mean(GLEU_val_emb))
    except:
        continue
print("GELU Score = ",np.mean(GLEU_val_emb))

(13187, 2)


1it [00:06,  6.34s/it]

GELU Score for  1  data point 0.9354838709677419


21it [02:13,  6.25s/it]

GELU Score for  21  data point 0.8085319589371981


...

GELU Score for  61  data point 0.8078555837400341


81it [08:29,  6.43s/it]

GELU Score for  81  data point 0.808525934806311


100it [10:30,  6.31s/it]

GELU Score =  0.8067715348990488


In [27]:
import warnings 
warnings.filterwarnings('ignore')
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm
import numpy as np
from nltk.translate.gleu_score import sentence_gleu


GLEU_val_emb = []
test_data = df_test1
print(df_test1.shape)

for ind,i in tqdm(test_data.iterrows(),position=0):
    try:
        text = str(i.enc_input)
        #pred = predict(str(i.enc_input),model)[0].split()
        pred = correct_grammar(text, num_return_sequences=1)[0]
        l = len(text)
        p = len(pred)
        if(p>l):
            pred = pred[:l]
            
        act = [str(i.dec_input).split()]
        pred_s = str(pred).split()
        b = sentence_gleu(act,pred_s)
        GLEU_val_emb.append(b)
        if(ind%1000 ==0):
            print("GELU Score for ",ind+1," data point",np.mean(GLEU_val_emb))
    except:
        continue
print("GELU Score = ",np.mean(GLEU_val_emb))

(13187, 2)


1it [00:01,  1.34s/it]

GELU Score for  1  data point 0.9354838709677419


1001it [22:02,  1.30s/it]

GELU Score for  1001  data point 0.8039029214744148


...

GELU Score for  12001  data point 0.810914551153518


13001it [4:39:33,  1.26s/it]

GELU Score for  13001  data point 0.8095086869544634


13187it [4:43:27,  1.29s/it]

GELU Score =  0.8095147417782044
